rules of the game : https://cdn.1j1ju.com/medias/85/5e/7c-pickomino-rulebook.pdf

subject : https://team.inria.fr/polaris/files/2023/10/project_2023.pdf

#PART 1: MDP for 1 player

In [1]:
import random as rd
import itertools
import math

from dice_state import *
from mdp import *

\go faire un truc déguelasse avec dico et sets et ensuite on optimise si besoin

on considère dans toute la suite que 6 est synonyme de 'W' (worm)

a state is : a sorted list of dices rolled + a set of faces you cannot choose + the sum of dices already picked

In [2]:
from dice_state import *
from mdp import *

c = 0 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
# dices = (1, 2)
score = 0
used = set()
state = DiceState(dices, score, used)

mdp.explore(state)

print(state, mdp.value[state], mdp.opti[state])


(1, 3, 3, 3, 3, 4, 4, 5, 6) 0 set() 1.232827668101253 Continue with dice 4


In [3]:
c = -3 #value if you lose the turn
r = [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4] + [4]*100
mdp = MDP(c,r)

dices = (1, 3, 3, 3, 3, 4, 4, 5, 6)
score = 0
used = set()

state = DiceState(dices, score, used)
mdp.explore(state)


print(state, mdp.value[state], mdp.opti[state])


(1, 3, 3, 3, 3, 4, 4, 5, 6) 0 set() 0.2562805612151674 Continue with dice 4


In [4]:
from mdp import *
from dice_state import *

c = 0 #value if you lose the turn
r = [0,0,0,1] + [0]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

print(len(mdp.value))
"""for i in mdp.value.keys():
    print(i, mdp.value[i])"""

0
1
2
3
4
5
0
1
2
3
4
5
0.24243906674019267
68572


'for i in mdp.value.keys():\n    print(i, mdp.value[i])'

In [5]:
c = 0 #value if you lose the turn
r = [0,0,0,0,0,0,1] + [1]*100
mdp = MDP(c,r)

mdp.explore_all()

p = mdp.compute_value_total()
print(p)

0
1
2
3
4
5
0
1
2
3
4
5
0.310248006941


In [6]:
it = itertools.combinations_with_replacement([1, 1, 3, 4, 5, 6], 2)
for i in it:
    print(i)

(1, 1)
(1, 1)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(1, 1)
(1, 3)
(1, 4)
(1, 5)
(1, 6)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(4, 4)
(4, 5)
(4, 6)
(5, 5)
(5, 6)
(6, 6)


In [7]:
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *
from alpha_beta_player import AlphaBetaPlayer


p1 = AlphaBetaPlayer(0, 2)
n1 = f"AlphaBeta(0, 2)"
p2 = AlphaBetaPlayer(1,1)
n2 = f"AlphaBeta(1, 1)"

NUM_GAMES = 100

w, t = 0, 0
for i in range(NUM_GAMES):
    if i % 2 == 0:
        winner = run_game(p1, p2)
    else:
        winner = run_game(p2, p1)
    if winner == PlayerTurn.PLAYER_1:
        w+=1
    t += 1
    print(f"[{i}] {n1} wins with probabilty {w/t} over {n2}")



[0] AlphaBeta(0, 2) wins with probabilty 0.0 over AlphaBeta(1, 1)
[1] AlphaBeta(0, 2) wins with probabilty 0.0 over AlphaBeta(1, 1)
[2] AlphaBeta(0, 2) wins with probabilty 0.0 over AlphaBeta(1, 1)
[3] AlphaBeta(0, 2) wins with probabilty 0.25 over AlphaBeta(1, 1)
[4] AlphaBeta(0, 2) wins with probabilty 0.4 over AlphaBeta(1, 1)
[5] AlphaBeta(0, 2) wins with probabilty 0.5 over AlphaBeta(1, 1)
[6] AlphaBeta(0, 2) wins with probabilty 0.42857142857142855 over AlphaBeta(1, 1)
[7] AlphaBeta(0, 2) wins with probabilty 0.5 over AlphaBeta(1, 1)
[8] AlphaBeta(0, 2) wins with probabilty 0.5555555555555556 over AlphaBeta(1, 1)
[9] AlphaBeta(0, 2) wins with probabilty 0.5 over AlphaBeta(1, 1)
[10] AlphaBeta(0, 2) wins with probabilty 0.45454545454545453 over AlphaBeta(1, 1)
[11] AlphaBeta(0, 2) wins with probabilty 0.4166666666666667 over AlphaBeta(1, 1)
[12] AlphaBeta(0, 2) wins with probabilty 0.46153846153846156 over AlphaBeta(1, 1)
[13] AlphaBeta(0, 2) wins with probabilty 0.5 over AlphaBeta

In [1]:
from alpha_beta_player import AlphaBetaPlayer
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *

p1 = AlphaBetaPlayer(1,1)
p2 = AlphaBetaPlayer(1.5,0.5)

w = run_game(p1,p2)
print(w)

according to dico
(1, 1, 1, 4, 4, 5, 6, 6) 0 set() Continue with dice 6
PlayerTurn.PLAYER_1's turn:
Dice State: (1, 1, 1, 4, 4, 5, 6, 6) 0 set()
Grid State: 
U U U U U U U U U U U U U U U U 
1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 
Player 1 Tiles: []
Player 2 Tiles: []

Continue with dice 6

according to dico
(2, 2, 2, 2, 4, 6) 10 {6} Continue with dice 4
PlayerTurn.PLAYER_1's turn:
Dice State: (2, 2, 2, 2, 4, 6) 10 {6}
Grid State: 
U U U U U U U U U U U U U U U U 
1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 
Player 1 Tiles: []
Player 2 Tiles: []

Continue with dice 4

according to dico
(1, 2, 3, 4, 6) 14 {4, 6} Continue with dice 3
PlayerTurn.PLAYER_1's turn:
Dice State: (1, 2, 3, 4, 6) 14 {4, 6}
Grid State: 
U U U U U U U U U U U U U U U U 
1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 
Player 1 Tiles: []
Player 2 Tiles: []

Continue with dice 3

according to dico
(2, 2, 6, 6) 17 {3, 4, 6} Continue with dice 2
PlayerTurn.PLAYER_1's turn:
Dice State: (2, 2, 6, 6) 17 {3, 4, 6}
Grid State: 
U U U U U U U U U U U U U U U

In [9]:
from alpha_beta_player import AlphaBetaPlayer
from game_state import GameState, TileState
from dice_state import DiceState
from move import Move, MoveType
from player import GreedyPlayer
from run_game import *
from mdp import *

m = MDP(0, [1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4])

state = DiceState((2, 4, 6, 6, 6), 15, {4})

m.explore(state)

print(m.opti[state])
for i in m.opti:
    print(i,m.opti[i])

Continue with dice 6
() 21 {1, 2, 4} Lose
(1, 1, 1, 1) 17 {2, 4} Continue with dice 1
(1,) 20 {1, 2, 4} Lose
(2,) 20 {1, 2, 4} Lose
() 23 {1, 2, 3, 4} Lose
(3,) 20 {1, 2, 4} Continue with dice 3
(4,) 20 {1, 2, 4} Lose
() 25 {1, 2, 4, 5} Lose
(5,) 20 {1, 2, 4} Continue with dice 5
() 25 {1, 2, 4, 6} Stop and pick tile 4
(6,) 20 {1, 2, 4} Continue with dice 6
(1, 1, 1, 2) 17 {2, 4} Continue with dice 1
(1, 1, 1) 20 {2, 3, 4} Continue with dice 1
(1,) 22 {1, 2, 3, 4} Lose
(2,) 22 {1, 2, 3, 4} Lose
(3,) 22 {1, 2, 3, 4} Lose
(4,) 22 {1, 2, 3, 4} Lose
() 27 {1, 2, 3, 4, 5} Lose
(5,) 22 {1, 2, 3, 4} Continue with dice 5
() 27 {1, 2, 3, 4, 6} Stop and pick tile 6
(6,) 22 {1, 2, 3, 4} Continue with dice 6
(1, 1, 2) 20 {2, 3, 4} Continue with dice 1
(1, 1, 3) 20 {2, 3, 4} Continue with dice 1
(1, 1, 4) 20 {2, 3, 4} Continue with dice 1
(1, 1) 25 {2, 3, 4, 5} Continue with dice 1
(1,) 26 {1, 2, 3, 4, 5} Lose
(2,) 26 {1, 2, 3, 4, 5} Lose
(3,) 26 {1, 2, 3, 4, 5} Lose
(4,) 26 {1, 2, 3, 4, 5} Lose
(5

In [1]:
from ucb import *
from alpha_beta_player import AlphaBetaPlayer


p1 = AlphaBetaPlayer(1,1)

num_games = 1000  
grid_size = 5

best_pair = simulate_games_ucb(p1, grid_size, num_games)
print(f"The best pair against (1,1) is: {best_pair}")

Playing pair (0.0, 0.0), with a winrate of 0.0
Playing pair (0.0, 0.0), with a winrate of 0.5
Playing pair (0.0, 0.5), with a winrate of 0.0
Playing pair (0.0, 1.0), with a winrate of 0.0
Playing pair (0.0, 1.5), with a winrate of 0.0
Playing pair (0.0, 2.0), with a winrate of 0.0
Playing pair (0.5, 0.0), with a winrate of 0.0
Playing pair (0.5, 0.5), with a winrate of 0.0
Playing pair (0.5, 1.0), with a winrate of 0.0
Playing pair (0.5, 1.5), with a winrate of 0.0
Playing pair (0.5, 2.0), with a winrate of 0.0
Playing pair (1.0, 0.0), with a winrate of 0.0
Playing pair (1.0, 0.5), with a winrate of 0.0
Playing pair (1.0, 1.0), with a winrate of 0.0
Playing pair (1.0, 1.5), with a winrate of 0.0
Playing pair (1.0, 2.0), with a winrate of 0.0
Playing pair (1.5, 0.0), with a winrate of 0.0
Playing pair (1.5, 0.5), with a winrate of 0.0
Playing pair (1.5, 1.0), with a winrate of 0.0
Playing pair (1.5, 1.5), with a winrate of 0.0
Playing pair (1.5, 2.0), with a winrate of 0.0
Playing pair 